In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

In [3]:
data = pd.read_csv('dataset.csv')
data.drop(['Unnamed: 0'],axis=1)

,Label,Tweet
0,1,"""RT @NPR: One doctor in Hong Kong says he unde..."
1,1,"""RT @NPR: One doctor in Hong Kong says he unde..."
2,1,RT @ABCWorldNews: Hundreds of Hong Kong hospit...
3,1,RT @ABCWorldNews: Hundreds of Hong Kong hospit...
4,1,RT @Independent: Hong Kong hospital workers st...
...,...,...
221,0,RT @NomenChanX: @ReutersWorld Amazon se retira...
222,0,RT @raythemouse: Does this image show scale of...
223,0,RT @ReutersBiz: Amazon withdraws from Barcelon...
224,0,RT @siva217: Watch @AgendaFreeTV's broadcast: ...
